In [ ]:
'''
Seth Tyler
ISD503 Project
Fall 2021
'''

In [1]:
# This script is used to generate new datasets that only include a specific number of launches (i.e. only the first 5 launches of all vehicles)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import math
from scipy.special import gamma, btdtri
import scipy.stats as ss

In [2]:
verbose = 0

In [3]:
datafile = '../08-Datasets/Aggregated_SLR_Dataset_v4.xlsx'
sheet = 'All-exclusionsandfiltered'

df = pd.read_excel(datafile, sheet_name=sheet,engine="openpyxl") #if using .xlsx
# df = pd.read_excel(datafile, sheet_name=sheet, engine='xlrd') #if using .xls

In [6]:
def first_launches(df, launch_number=20):
    fun_df = pd.DataFrame()
    fun_df = df.copy()
    fun_df['pandas_running_successes'] = fun_df['Success_Bool'].cumsum()
    fun_df['pandas_running_failures'] = fun_df['Failure_Bool'].cumsum()
    fun_df['pandas_running_launches'] = fun_df['pandas_running_failures']+fun_df['pandas_running_successes']
    successes = fun_df['pandas_running_successes'].loc[fun_df['pandas_running_launches'] == launch_number].item()
    failures = fun_df['pandas_running_failures'].loc[fun_df['pandas_running_launches'] == launch_number].item()
#     print(rslt_df['pandas_running_launches'])
    #print(rslt_df['pandas_running_launches'].loc[rslt_df['pandas_running_launches'] <= launch_number])
    filtered_df = fun_df.loc[fun_df['pandas_running_launches'] == launch_number]
#     print(filtered_df)
    return filtered_df

In [7]:
launch_numbers=[1,2,3,4,5,6,7]

for launch_number in launch_numbers:
    candidate_vhcl_fam_list = []
    interim_df = pd.DataFrame()
    final_df = pd.DataFrame(columns=['Date', 'Vehicle', 'ID', 'Payload/Mission', 'Site', 'Orbit',
           'Mass (orig)', 'Mass_for_hist', 'Mass_Cat', 'Failure_Text',
           'Clean_Veh_Name', 'Veh_Family', 'Veh_Mfctr', 'Veh_Mfctr_Region',
           'Launch_Country', 'Launch_Site', 'Veh_Active', 'Veh_SubFam_Active',
           'Veh_GenFam_Active', 'Failure_Bool', 'Success_Bool',
           'Failure_Running_Total', 'Success_Running_Total', 'Launch_Total',
           'Gen_Veh_Family', 'Veh_Generation', 'Number_of_Stages',
           'Veh_Generation_Category', 'Number_of_Stages_Category',
           'pandas_running_successes', 'pandas_running_failures',
           'pandas_running_launches'])


    Veh_Family=df['Veh_Family'].unique()
    Veh_Family.sort()

    for vhcl_fam in Veh_Family:
        if verbose: print('vhcl=',vhcl_fam)
        rslt_df = df.loc[df['Veh_Family']==vhcl_fam]

        rslt_df.set_index('Date')

        total_successes = max(rslt_df['Success_Bool'].cumsum())
        total_failures = max(rslt_df['Failure_Bool'].cumsum())
        total_launches = total_successes + total_failures

        if total_launches < launch_number:
            if verbose: print(vhcl_fam,'has too few launches')
        else:
            candidate_vhcl_fam_list.append(vhcl_fam)

    print('Number of vehicles in the running is',str(len(candidate_vhcl_fam_list)))
    # print(candidate_vhcl_fam_list)

    for vhcl_fam in candidate_vhcl_fam_list:
    #     print('working on', vhcl_fam)
        rslt_df = df.loc[df['Veh_Family']==vhcl_fam]
        interim_df = first_launches(rslt_df, launch_number=launch_number)
        final_df = final_df.append(interim_df)#, ignore_index=True, sort=False)

    # print(final_df)
    filename = '../08-Datasets/filtered_to_'+str(launch_number)+'_launches.xlsx'
    # To write all the file:
    # final_df.to_excel(filename, sheet_name='All')

    excel_df = pd.DataFrame()
    excel_df= final_df[['Mass_Cat','Veh_Generation_Category','Number_of_Stages_Category', 'Failure_Bool']].copy()
    excel_df.to_excel(filename,sheet_name='ForNeticaShortExcluded')


Number of vehicles in the running is 67
Number of vehicles in the running is 52
Number of vehicles in the running is 44
Number of vehicles in the running is 36
Number of vehicles in the running is 32
Number of vehicles in the running is 28
Number of vehicles in the running is 27
